<a href="https://colab.research.google.com/github/saspav/DA-104/blob/main/%D0%9F%D0%B0%D0%B2%D0%BB%D0%BE%D0%B2%D0%B0_%D0%A1%D0%92_%D0%A2%D1%80%D0%B5%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%BE%D1%87%D0%BD%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Задание 1. Собираем данные с помощью Requests**

С помощью методов API ВКонтакте получите список высших учебных заведений и названия их факультетов в городе Томск.
Результат должен быть записан в файл JSON в следующем формате:

In [1]:
# {
#   "cities": [
#     {
#         "id": <ID города>,
#         "name": <Название города>,
#         "universites": [
#             {
#               "id": <ID ВУЗа>,
#               "name": <Название ВУЗа>
#               "faculties": [<Название факультета>, …]
#             },
#             ...
#         ]
#     },
#     ...
# }
!pip install vk_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 1.6 MB/s 


In [2]:
import requests
import json

ACCESS_TOKEN = "98a4c4e1e6f40a38d0f03b179c68e7a4a99d5837e9ea0822524c204b9fccf07c4848e9b6c02d67d95004e"
URL_COUNTRY = "https://api.vk.com/method/database.getCountries"
URL_CITY = "https://api.vk.com/method/database.getCities"
URL_VUZ = "https://api.vk.com/method/database.getUniversities"
URL_FAC = "https://api.vk.com/method/database.getFaculties"

CONST_PARAMS = {"access_token": ACCESS_TOKEN, "v": "5.131"}


def handle_response(resp, item_name):
    if resp.get("error"):
        print(f"Get {item_name} error!")
        return

    items = resp['response']['items']
    if items:
        return items
    print(f"No {item_name} was found!")


def get_country(code='RU'):
    """
    Получение страны по заданному коду
    """
    params = {**CONST_PARAMS, "code": code, "count": 1}

    country = handle_response(requests.get(URL_COUNTRY, params=params).json(),
                              "country")
    return country[0] if country else None


def get_cities(country_id, q='', count=1):
    """
    Получение города по заданной стране
    """
    params = {**CONST_PARAMS, "country_id": country_id, "count": count, 'q': q}

    cities = handle_response(requests.get(URL_CITY, params=params).json(),
                             "cities")
    return cities if cities else None


def get_universities(city_id):
    """
    Получение названий высших учебных заведений в заданном городе
    """
    params = {**CONST_PARAMS, "city_id": city_id}

    univers = handle_response(requests.get(URL_VUZ, params=params).json(),
                              "universities")
    return [(unv['id'], unv['title'].strip()) for unv in univers] if univers else []


def get_faculties(university_id):
    """
    Получение названий факультетов в заданном ВУЗе
    """
    params = {**CONST_PARAMS, "university_id": university_id}
    faculties = handle_response(requests.get(URL_FAC, params=params).json(),
                                "faculties")
    return sorted(fac['title'] for fac in faculties) if faculties else []


def main_procedure():
    # Получаем список городов страны
    city = get_cities(get_country().get('id'), 'Томск')[0]
    if not city:
        return

    # Получаем список высших учебных заведений в городе
    univers = get_universities(city['id'])
    universites = []
    for univer_id, univer_name in univers:
        universites.append({"id": univer_id,
                            "name": univer_name,
                            "faculties": get_faculties(univer_id)})
    city_data = [{"id": city['id'],
                  "name": city['title'],
                  "universites": universites
                  }]
    results = dict(cities=city_data)

    # Записываем все в JSON файл
    with open("universites.json", "w") as file:
        json.dump(results, file, ensure_ascii=False)


main_procedure()

No faculties was found!
No faculties was found!
No faculties was found!
No faculties was found!


# **Задание 2. Библиотека VK API**

Напишите код, который получает список названия школ города Кемерово с помощью библиотеки vk_api и записывает результаты в файл JSON.

In [3]:
import vk_api
import json

ACCESS_TOKEN = "98a4c4e1e6f40a38d0f03b179c68e7a4a99d5837e9ea0822524c204b9fccf07c4848e9b6c02d67d95004e"


# Инициализация сессии с помощью токена доступа
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

# Получение ID города Кемерово
city = vk.database.getCities(country_id=1, q='Кемерово', count=1)
city_id = city['items'][0]['id'] if city['count'] else None
city_title = city['items'][0]['title'] if city['count'] else None
# Получение школ города Кемерово
schools = vk.database.getSchools(city_id=city_id)
# Формирование списка школ
schools_title = [s['title'] for s in schools['items']]
# Формирование словаря для записи в файл
city_data = [{"id": city_id,
              "name": city_title,
              "schools": sorted(schools_title)
              }]
results = dict(city=city_data)
# Записываем все в JSON файл
with open("schools.json", "w") as file:
    json.dump(results, file, ensure_ascii=False)

# **Задание 3. Запись данных в CSV формат**

С помощью методов API «ВКонтакте» получите 1000 подписчиков группы «Лентач», отсортирванных по дате регистрации.

Вам необходимо собрать следующие данные в CSV файл: пол, название город, семейное положение (ФИО партнера не указывать).

In [4]:
import vk_api
import csv

ACCESS_TOKEN = '98a4c4e1e6f40a38d0f03b179c68e7a4a99d5837e9ea0822524c204b9fccf07c4848e9b6c02d67d95004e'

SEX = {1: 'женский', 2: 'мужской', 0: 'пол не указан'}
RELATION = {1: 'не женат/не замужем', 2: 'есть друг/есть подруга',
            3: 'помолвлен/помолвлена', 4: 'женат/замужем', 5: 'всё сложно',
            6: 'в активном поиске', 7: 'влюблён/влюблена',
            8: 'в гражданском браке', 0: 'не указано'}

# Инициализация сессии с помощью токена доступа
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

# Получение ID группы
group_search = vk.groups.search(q='Лентач', count=1)
id_lentach = group_search.get('items', [])[0].get('id')
# Получение списка подписчиков группы
users = []
counts = 200
for offset in range(0, 1001, counts):
    group_members = vk.groups.getMembers(group_id=id_lentach,
                                         count=counts,
                                         offset=offset,
                                         sort='id_asc',
                                         fields='sex, city, relation')

    # Формирование списка с нужными полями
    temp = [{'id_user': user['id'],
             'пол': SEX[int(user.get('sex', 0))],
             'название город': user.get('city', {'title': None}).get('title'),
             'семейное положение': RELATION[int(user.get('relation', 0))]}
            for user in group_members.get('items', [None])]
    users.extend(temp)

# Нам нужно ровно 1000 подписчиков
users = users[:1000]

# Запись данных в CSV файл
with open("group_members.csv", "w", encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=users[0].keys())
    writer.writeheader()
    for user in users:
        writer.writerow(user)

##### В CSV файл можно записать гораздо проще:

In [5]:
import pandas as pd

# Создаем ДФ
df = pd.DataFrame([user.values() for user in users], columns=users[0].keys())
# Сохраняем ДФ в файл
df.to_csv('df_group_members.csv', index=False)